In [1]:
import numpy as np
import seaborn as sns
import scipy.stats as stats
import matplotlib.pyplot as plt
from pandas import *
from numpy.linalg import inv
import random
import statistics
import time
import pylab

In [2]:
def printI(a):
    for row in range(len(a)):
        for col in range (len(a[row])):
            print("{:3.0f}".format(a[row][col]), end = " ")
        print()

def printF(a):
    for row in range(len(a)):
        for col in range (len(a[row])):
            print("{:8.5f}".format(a[row][col]), end = " ")
        print()

In [3]:
#SeedForYield = 13
#This cell produces all the Yield-Matrices (YM) Inverses(I) of the Yield Matrices
        
nC=6; nN=6; TotalStates=1211
States = np.loadtxt('StateDataL6.dat')

X = 0.1
SeedForYield = 13 #chosen any number
np.random.seed(SeedForYield)
YieldC = np.random.uniform(X,1-X,nC*nN)
YieldN = np.random.uniform(X,1-X,nC*nN)

M = np.zeros(shape=(nC,nN))
YM = np.zeros(shape=(TotalStates,nC+nN,nC+nN))
I = np.zeros(shape=(TotalStates,nC+nN,nC+nN))

for S in range(TotalStates):
    #print(S,States[S,:])
    for i in range(0,nC,1):
        for j in range(0,nN,1):
            M[i,j]=States[S,(i*nC)+j]
    #printI(M)
    
    IDC=[]
    IDN=[]
    k=0
    for i in range(0,nC,1):
        for j in range(0,nN,1):
            if(M[i,j] != 0):
                IDC.append(i)
                IDN.append(j)
                k+=1
                
    #print(IDC)
    #print(IDN)
    
    Y = np.zeros(shape=(nC+nN,nC+nN))
    for i in range(0,len(IDC), 1):
        Y[IDC[i],i] = 1/YieldC[nC*IDC[i] + IDN[i]]
        Y[nN+IDN[i],i] = 1/YieldN[nC*IDC[i] + IDN[i]]
        
    w = len(IDC)
    while (w < nC+nN):
        for i in range(0,nC,1):
            for j in range(0,nN,1):
                if(M[i,j] == -1):
                    break
            if(M[i][j] != -1):
                Y[i,w] = 1.0
                w += 1
            
        for j in range(0,nN,1):
            for i in range(0,nC,1):
                if(M[i,j] == 1):
                    break
            if(M[i][j] != 1):
                Y[nC+j,w] = 1.0
                w += 1
            
    #print("Yield is")
    #printF(Y)
    YM[S,:,:] = Y
    I[S,:,:]=inv(Y)
    #print("Inverse is")
    #printF(I[S,:,:])
    #print("YM is")
    #printF(YM[S,:,:])

In [4]:
print(YieldC)
print(YieldN)

[0.72216193 0.29003298 0.75942283 0.87259936 0.87808089 0.4627594
 0.58723397 0.72042121 0.61329068 0.67761458 0.12802922 0.33875958
 0.14680999 0.78564875 0.39828322 0.64387836 0.30502396 0.37806497
 0.10753022 0.38666703 0.85927535 0.27431921 0.35551309 0.83421791
 0.12552293 0.15206763 0.6038632  0.79905075 0.10697259 0.69726179
 0.75027294 0.16057396 0.62516427 0.50740976 0.48390671 0.86445932]
[0.10000963 0.29758296 0.66978614 0.35966564 0.32159709 0.65635636
 0.8348414  0.29558056 0.46646865 0.30239415 0.40346663 0.58363106
 0.71790301 0.154334   0.64886806 0.53860808 0.21038884 0.17900258
 0.29644728 0.22142933 0.84079558 0.64408401 0.29012714 0.5551082
 0.54530564 0.15818977 0.77176681 0.42425559 0.21589679 0.25273605
 0.49251211 0.6696195  0.88795077 0.7998292  0.49923335 0.185424  ]


In [5]:
RatioYield = YieldC/YieldN
statistics.stdev(RatioYield)

1.4876751633661305

In [6]:
#This cell chooses a random Phi and checks the feasibility of each state at that Phi and stores the feasible state
#in the List StatePresenceData
start_time = time.time()
TotalPoints = 1000000
PhiData = []
StatePresenceData = []
VolFromMonte = np.zeros(shape=TotalStates)    


for PhiPoint in range(TotalPoints*SeedForYield, TotalPoints*(SeedForYield+1), 1):
    #print("PhiPoint:", PhiPoint)
    q = []
    np.random.seed(PhiPoint)#This contains the seed
    
    Phi = np.random.uniform(10,1000,nC+nN)

    PhiData.append(Phi)
    
    #print("Phi:", Phi)
    for S in range(TotalStates):
        X = np.matmul(I[S,], Phi)
        for i in range(nC+nN):
            if (X[i] < 0.0):
                break
        if (X[i] > 0.0):
            #print("State", S)
            q += [S]
            #Data[PhiPoint][nC+nN+S] = 1
            VolFromMonte[S] += 1
    StatePresenceData.append(q)
#printF(Data)
print(VolFromMonte)
#print(StatePresenceData)
print("--- %s seconds ---" % (time.time() - start_time))

[1440.  408.  272. ...  503.    2.   10.]
--- 2427.2849667072296 seconds ---


In [8]:
np.shape(StatePresenceData)

(1000000,)

In [9]:
np.shape(PhiData)

(1000000, 12)

In [13]:
PhiData[0]

array([968.99177874, 589.06408321, 766.98119002, 920.8469091 ,
       882.46080513, 864.17941879, 424.17052118, 754.55299841,
       234.67899639, 409.08292676, 688.56593935, 658.70138135])

In [10]:
np.savetxt('StatePresenceData_Seed13_1MillionFlux.txt', StatePresenceData, delimiter=' ', fmt='%s')

In [11]:
np.savetxt('PhiData_Seed13_1MillionFlux.txt', PhiData, delimiter=' ', fmt='%s')